### Import basic libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

### Load in the dataframe

In [2]:
df = pd.read_pickle('./main_df.pkl')
print(df.shape)
df.head()

(2213180, 14)


,virus,host,blastn,crisprdetect-2mismatch,gc_content,k25,k6-chebyshev,k6-kendalltau,k6-manhattan,piler-2mismatch,wish,group,group_code,y
0,NC_010363,NC_008527,60.8,0.0,0.021973,0.0,0.002122,0.398421,0.382144,0.0,-1.33553,Streptococcaceae,41,1
1,NC_010363,NC_002662,59.0,0.0,0.016709,0.0,0.001929,0.397773,0.377498,0.0,-1.33035,Streptococcaceae,41,1
2,NC_010363,NC_017949,59.0,0.0,0.020818,0.0,0.002088,0.396969,0.379686,0.0,-1.33341,Streptococcaceae,41,1
3,NC_010363,NC_017492,59.0,0.0,0.022209,0.0,0.002131,0.396148,0.380930,0.0,-1.33767,Streptococcaceae,41,1
4,NC_010363,NC_009004,59.0,0.0,0.020871,0.0,0.002088,0.397095,0.379834,0.0,-1.33341,Streptococcaceae,41,1


## Data preprocessing

### Get all of positive cases + the same number of negative cases randomly

In [3]:
# get all the positive cases
learning_df = df[df['y'] == 1]
# get the same amount of negative cases RANDOMLY
negative_learning_df = df[df['y'] == 0].sample(n=len(df[df['y'] == 1].index))

learning_df = pd.concat([learning_df, negative_learning_df], ignore_index=True)
print(learning_df.shape)
learning_df.head()

(33514, 14)


,virus,host,blastn,crisprdetect-2mismatch,gc_content,k25,k6-chebyshev,k6-kendalltau,k6-manhattan,piler-2mismatch,wish,group,group_code,y
0,NC_010363,NC_008527,60.8,0.0,0.021973,0.0,0.002122,0.398421,0.382144,0.0,-1.33553,Streptococcaceae,41,1
1,NC_010363,NC_002662,59.0,0.0,0.016709,0.0,0.001929,0.397773,0.377498,0.0,-1.33035,Streptococcaceae,41,1
2,NC_010363,NC_017949,59.0,0.0,0.020818,0.0,0.002088,0.396969,0.379686,0.0,-1.33341,Streptococcaceae,41,1
3,NC_010363,NC_017492,59.0,0.0,0.022209,0.0,0.002131,0.396148,0.380930,0.0,-1.33767,Streptococcaceae,41,1
4,NC_010363,NC_009004,59.0,0.0,0.020871,0.0,0.002088,0.397095,0.379834,0.0,-1.33341,Streptococcaceae,41,1


### Groups dristribution

### Encode categorical values

*Note: encoding not needed because training is not based on virus and host names*

In [6]:
'''
transformed_data = pd.get_dummies(filled_df, columns=['virus', 'host'])
transformed_data
'''

"\ntransformed_data = pd.get_dummies(filled_df, columns=['virus', 'host'])\ntransformed_data\n"

### Extract X and y arrays

\+ get groups for LeaveOneGroupOut

In [4]:
# can also use .values for X and y for speed, but without it it's easier to look at these sets 
X = learning_df.drop(['virus', 'host', 'group', 'group_code', 'y'], axis=1)
y = learning_df['y']
groups = learning_df['group_code'].values

print(f'shape of X: {X.shape}')
print(f'len(y): {len(y)}')

shape of X: (33514, 9)
len(y): 33514


# Training using cross validation

### Using cross_validate function

In [5]:
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

# determine the scoring method
scoring = ['f1']
# create logo cv procedure
logo = LeaveOneGroupOut()
# create model
model = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state=1)
# evaluate model
results = cross_validate(model, X, y, scoring=scoring, 
                        cv=logo, groups=groups, n_jobs=-1, return_estimator=True)

In [11]:
results.keys()

dict_keys(['fit_time', 'score_time', 'estimator', 'test_f1'])

In [12]:
results['test_f1']

array([1.        , 0.66666667, 0.08333333, 0.7       , 0.        ,
       0.60208644, 0.6       , 0.        , 0.66666667, 0.55060729,
       0.29787234, 0.63157895, 0.03030303, 0.81632653, 0.        ,
       0.        , 0.54042517, 0.71929825, 0.70588235, 1.        ,
       0.78787879, 0.58823529, 0.9205807 , 0.        , 0.58385093,
       0.87972509, 0.88888889, 0.66666667, 0.3       , 0.        ,
       0.94845361, 0.94845361, 0.15384615, 0.225     , 0.4057971 ,
       0.86956522, 0.5       , 1.        , 1.        , 0.        ,
       0.89144737, 0.85314685, 0.54545455, 0.76470588, 0.        ,
       0.5       , 0.41901408, 0.57657658])

In [13]:
len(results['estimator'])

48

## Get probabilities of class assignment for each element in main_df

for each estimator:
1. take the viruses from the test part of the estimator
2. for each of this virus estimate class for ALL the bacteria in main_df
3. check if estimation was correct
4. take best predictions (highest probabilities) for each virus
5. write these predictions down

### For main df:

Create X, y and groups sets for the main df

In [6]:
X_main = df.drop(['virus', 'host', 'group', 'group_code', 'y'], axis=1)
y_main = df['y']
groups_main = df['group_code'].values

Check if each subgroup contains all the hosts

In [7]:
hosts_num = len(pd.unique(df.loc[:, 'host']))
for i in range(0, max(groups_main)+1):
    if len(pd.unique(df.loc[groups_main == i, 'host'])) != hosts_num:
        print(f'ERROR: not enough hosts in subgroup {i} (hosts: {len(pd.unique(df.loc[groups_main == i, "host"]))})!')

Calculate the probabilities of classification for viruses in each of the test sets (i.e. where groups_main == i)

In [8]:
prob_df_main = pd.DataFrame(index=range(len(df['y'])), columns=['0', '1'])
prob_df_main['0'] = prob_df_main['0'].astype('float')
prob_df_main['1'] = prob_df_main['1'].astype('float')
for i in range(0, max(groups_main)+1):
    mask_main = groups_main == i
    prob_df_main.loc[mask_main, ['0', '1']] = results['estimator'][i].predict_proba(X_main.loc[mask_main,:])

Create a df with the results

In [9]:
df_all = pd.concat([df, prob_df_main], axis=1)

In [10]:
df_all.head()

,virus,host,blastn,crisprdetect-2mismatch,gc_content,k25,k6-chebyshev,k6-kendalltau,k6-manhattan,piler-2mismatch,wish,group,group_code,y,0,1
0,NC_010363,NC_008527,60.8,0.0,0.021973,0.0,0.002122,0.398421,0.382144,0.0,-1.33553,Streptococcaceae,41,1,0.6,0.4
1,NC_010363,NC_002662,59.0,0.0,0.016709,0.0,0.001929,0.397773,0.377498,0.0,-1.33035,Streptococcaceae,41,1,0.5,0.5
2,NC_010363,NC_017949,59.0,0.0,0.020818,0.0,0.002088,0.396969,0.379686,0.0,-1.33341,Streptococcaceae,41,1,0.6,0.4
3,NC_010363,NC_017492,59.0,0.0,0.022209,0.0,0.002131,0.396148,0.380930,0.0,-1.33767,Streptococcaceae,41,1,0.6,0.4
4,NC_010363,NC_009004,59.0,0.0,0.020871,0.0,0.002088,0.397095,0.379834,0.0,-1.33341,Streptococcaceae,41,1,0.6,0.4


Check if there indeed are all the expected predictions (2699 hosts for each virus)

In [11]:
hosts_num = len(pd.unique(df['host']))
for i, el in enumerate(df_all.groupby('virus')['host'].count()):
    if el != hosts_num:
        print(f'Error for virus {i}')

Determine if the predition was correct

In [12]:
df_all['estimator_correct'] = np.nan
df_all['estimator_correct'] = df_all['estimator_correct'].astype(bool)
mask = df_all['y'] == 0
df_all.loc[mask,'estimator_correct'] = df_all['0'] > df_all['1']
df_all.loc[mask, 'prob'] = df_all['0']
mask = df_all['y'] == 1
df_all.loc[mask, 'estimator_correct'] = df_all['1'] > df_all['0']
df_all.loc[mask, 'prob'] = df_all['1']
df_all['prob'] = df_all['prob'].astype('float')

In [38]:
correct_df = df_all[df_all['estimator_correct'] == True]
df_all

,virus,host,blastn,crisprdetect-2mismatch,gc_content,k25,k6-chebyshev,k6-kendalltau,k6-manhattan,piler-2mismatch,wish,group,group_code,y,0,1,estimator_correct,prob
6,NC_010363,NC_012438,57.2,0.0,0.009055,0.0,0.002532,0.515620,0.588794,0.0,-1.35836,Streptococcaceae,41,0,0.8,0.2,True,0.8
7,NC_010363,NC_006814,53.6,0.0,0.010561,0.0,0.001443,0.445837,0.412691,0.0,-1.34041,Streptococcaceae,41,0,1.0,0.0,True,1.0
8,NC_010363,NC_021181,53.6,0.0,0.010478,0.0,0.001445,0.445844,0.412664,0.0,-1.34044,Streptococcaceae,41,0,1.0,0.0,True,1.0
9,NC_010363,NC_021721,53.6,0.0,0.125869,0.0,0.003324,0.691869,0.655933,0.0,-1.37431,Streptococcaceae,41,0,1.0,0.0,True,1.0
10,NC_010363,NC_018420,53.6,0.0,0.047558,0.0,0.008447,0.441399,0.475200,0.0,-1.35649,Streptococcaceae,41,0,0.8,0.2,True,0.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2213175,NC_024392,NC_009664,0.0,0.0,0.380006,0.0,0.006918,1.525475,1.535981,0.0,-1.46593,Listeriaceae,22,0,1.0,0.0,True,1.0
2213176,NC_024392,NC_011891,0.0,0.0,0.382821,0.0,0.010627,1.509128,1.523096,0.0,-1.47323,Listeriaceae,22,0,1.0,0.0,True,1.0
2213177,NC_024392,NC_015514,0.0,0.0,0.382899,0.0,0.007646,1.544129,1.575653,0.0,-1.45837,Listeriaceae,22,0,1.0,0.0,True,1.0
2213178,NC_024392,NC_011145,0.0,0.0,0.384063,0.0,0.010793,1.507242,1.526537,0.0,-1.47286,Listeriaceae,22,0,1.0,0.0,True,1.0


#### Determine best hosts for each virus - ones with the highest probabilities (and correct classification) 

Get indecies for the viruses with highest '1' values (for each virus)

In [13]:
# This means: groupby viruses, within these groups find rows with max values in '1' and return the indecies
mask = df_all.groupby('virus')['1'].transform(max) == df_all['1']
df_all[mask]

,virus,host,blastn,crisprdetect-2mismatch,gc_content,k25,k6-chebyshev,k6-kendalltau,k6-manhattan,piler-2mismatch,wish,group,group_code,y,0,1,estimator_correct,prob
12,NC_010363,NC_023063,51.8,0.0,0.039944,0.0,0.002440,0.472651,0.489261,0.0,-1.35088,Streptococcaceae,41,0,0.4,0.6,False,0.4
225,NC_013599,NC_010513,9281.0,0.0,0.048394,4917.0,0.001123,0.466373,0.343648,0.0,-1.37070,Xanthomonadaceae,47,1,0.0,1.0,True,1.0
226,NC_013599,NC_004556,8473.0,0.0,0.049805,8718.0,0.001119,0.468682,0.345848,0.0,-1.37728,Xanthomonadaceae,47,1,0.0,1.0,True,1.0
227,NC_013599,NC_010577,8473.0,0.0,0.050001,8747.0,0.001118,0.469334,0.346064,0.0,-1.37748,Xanthomonadaceae,47,1,0.0,1.0,True,1.0
228,NC_013599,NC_017562,8471.0,0.0,0.049755,8565.0,0.001122,0.469839,0.346372,0.0,-1.37730,Xanthomonadaceae,47,1,0.0,1.0,True,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2177840,NC_024367,NC_015637,0.0,0.0,0.051702,0.0,0.001876,0.813533,0.620529,0.0,-1.40234,Rhodobacteraceae,36,0,0.1,0.9,False,0.1
2177841,NC_024367,NC_022224,0.0,0.0,0.051724,0.0,0.001882,0.813859,0.620627,0.0,-1.40237,Rhodobacteraceae,36,0,0.1,0.9,False,0.1
2177875,NC_024367,NC_017964,0.0,0.0,0.056229,0.0,0.001968,0.772447,0.626898,0.0,-1.40305,Rhodobacteraceae,36,0,0.1,0.9,False,0.1
2184873,NC_024375,NC_012913,0.0,0.0,0.059733,0.0,0.002169,0.640354,0.544694,0.0,-1.38892,Listeriaceae,22,0,0.1,0.9,False,0.1


In [14]:
df_1 = df_all.loc[mask, ['virus', 'host', '1']]
df_1

,virus,host,1
12,NC_010363,NC_023063,0.6
225,NC_013599,NC_010513,1.0
226,NC_013599,NC_004556,1.0
227,NC_013599,NC_010577,1.0
228,NC_013599,NC_017562,1.0
...,...,...,...
2177840,NC_024367,NC_015637,0.9
2177841,NC_024367,NC_022224,0.9
2177875,NC_024367,NC_017964,0.9
2184873,NC_024375,NC_012913,0.9


### Write down the classification results

Load in the taxonomy JSONs

In [15]:
import json
import pathlib

orgs = {}
for file in pathlib.Path('./taxonomy/').iterdir():
    with open(file, 'r') as open_file:
        orgs[file.stem] = json.load(open_file)
        
orgs.keys()

dict_keys(['host', 'virus'])

Get the names of viruses and hosts

In [16]:
df_1['virus_name'] = df_1.apply(lambda row: orgs["virus"][row['virus']]["organism_name"].split(",")[0], axis=1)
df_1['host_name'] = df_1.apply(lambda row: orgs["host"][row['host']]["organism_name"], axis=1)

In [17]:
# reorder columns
df_1 = df_1[['virus', 'virus_name', 'host', 'host_name', '1']]

Sort the dataframe - for looking for differences later

In [18]:
df_1 = df_1.sort_values(by=['virus', 'host'])

Write result to file

In [43]:
df_1.to_csv('all_bacteria_with_highest_1_prob_log.tsv', sep='\t', index=False)

### Check if estimator predicted correct host species, genus, family, order ... 

Get 1 host for each virus (group)

In [26]:
df_chosen = df_1.groupby('virus').apply(pd.DataFrame.sample, 
                    n=1, axis=0).reset_index(drop=True)
df_chosen

,virus,virus_name,host,host_name,1
0,NC_000866,Enterobacteria phage T4,NC_009567,Haemophilus influenzae PittGG,1.0
1,NC_000871,Streptococcus phage Sfi19,NC_008525,Pediococcus pentosaceus ATCC 25745,1.0
2,NC_000872,Streptococcus phage Sfi21,NC_017594,Streptococcus salivarius 57.I,1.0
3,NC_000896,Lactobacillus prophage phiadh,NC_005362,Lactobacillus johnsonii NCC 533,1.0
4,NC_000902,Enterobacteria phage VT2-Sakai,NC_004431,Escherichia coli CFT073,1.0
...,...,...,...,...,...
815,NC_024387,Listeria phage LP-101,NC_021824,Listeria monocytogenes strain J2-064,1.0
816,NC_024388,Leuconostoc phage phiLN34,NC_017312,Enterococcus faecalis 62,0.8
817,NC_024389,Leuconostoc phage phiLNTR2,NC_018178,Melioribacter roseus P3M,0.9
818,NC_024391,Staphylococcus phage DW2,NC_017763,Staphylococcus aureus subsp. aureus HO 5096 0412,1.0


Determine if the chosen bacteria is a correct prediction at a given taxonomic rank

In [38]:
def lookup_taxonomy(x, rank):
    rank_idx = orgs['host'][x['host']]['lineage_ranks'].index(rank)
    return 1 if orgs["host"][x['host']]["lineage_names"][rank_idx] == \
        orgs['virus'][x['virus']]['host']['lineage_names'][rank_idx] else 0

In [40]:
ranks = ['species', 'genus', 'family', 'order', 'class', 'phylum', 'superkingdom']
for rank in ranks:
    df_chosen[f'{rank}_correct'] = df_chosen.apply(lookup_taxonomy, rank=rank, axis=1)

In [44]:
df_chosen.to_csv('1_bac_per_vir_log.tsv', sep='\t', index=False)

Calculate percentage of correct predictions on all taxonomic levels

In [45]:
# the log will contain a commented summary at the top
with open('1_bac_per_vir_log.tsv', mode='r+') as of:
    to_write = '# Prediction summary:\n'
    content = of.read()
    of.seek(0, 0)
    for rank in ranks:
        to_write += (f'# {rank}: {df_chosen[f"{rank}_correct"].sum() / len(df_chosen.index)*100:.2f}% correct\n')
    of.write(to_write + content)

In [46]:
df_chosen[df_chosen['1'] != 1]

,virus,virus_name,host,host_name,1,species_correct,genus_correct,family_correct,order_correct,class_correct,phylum_correct,superkingdom_correct
8,NC_001330,Enterobacteria phage alpha3,NC_022513,Ralstonia pickettii DTP0602 chromosome 1,0.9,0,0,0,0,0,1,1
9,NC_001331,Pseudomonas phage Pf1,NC_023066,Pseudomonas aeruginosa LES431,0.9,1,1,1,1,1,1,1
10,NC_001332,Enterobacteria phage I2-2,NC_015722,Candidatus Midichloria mitochondrii IricVA,0.9,0,0,0,0,0,1,1
11,NC_001341,Acholeplasma phage MV-L1,NC_020304,Desulfocapsa sulfexigens DSM 10523,0.7,0,0,0,0,0,0,1
14,NC_001417,Enterobacterio phage MS2,NC_020133,Mycobacterium liflandii 128FXT,0.8,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
810,NC_024379,Escherichia phage PE3-1,NC_012892,Escherichia coli BL21(DE3),0.8,1,1,1,1,1,1,1
811,NC_024381,Pseudomonas phage vB_PaeS_SCH_Ab26,NC_007510,Burkholderia sp. 383 chromosome 1,0.7,0,0,0,0,0,1,1
812,NC_024383,Listeria phage LP-083-2,NC_017728,Listeria monocytogenes 07PF0776,0.9,1,1,1,1,1,1,1
816,NC_024388,Leuconostoc phage phiLN34,NC_017312,Enterococcus faecalis 62,0.8,0,0,0,1,1,1,1


In [52]:
df_chosen['1'].describe()

count    820.000000
mean       0.923659
std        0.120950
min        0.300000
25%        0.900000
50%        1.000000
75%        1.000000
max        1.000000
Name: 1, dtype: float64

In [56]:
df_test = pd.read_csv('1_bac_per_vir_log.tsv', sep='\t', comment='#')

TODO: why do we have so many good predictions with such high probability?

# TODO: implement F1 + precision/recall curves for all estimators in cross validation

e.g. take estimator from cross_validate function + use from_estimator 

# Additional/old code

Explicit tqdm handling (better visualisaition)

*Watch out -* ***long!***

*This code is for LeaveOneOut* ***only!***

In [ ]:
'''
# FOR LEAVE ONE OUT ONLY!
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from tqdm import tqdm
# enumerate splits
y_true, y_pred = list(), list()
pbar = tqdm(total=len(X))

for train_ix, test_ix in cv.split(X):
    # split data
    X_train, X_test = X[train_ix, :], X[test_ix, :]
    y_train, y_test = y[train_ix], y[test_ix]
    # fit model
    model = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state=1)
    model.fit(X_train, y_train)
    # evaluate model
    yhat = model.predict(X_test)
    # store
    y_true.append(y_test[0])
    y_pred.append(yhat[0])
    pbar.update(1)
pbar.close()
# calculate accuracy
acc = accuracy_score(y_true, y_pred)
print(f'Accuracy: {acc:.3f}')
'''

### Is the prediction replicable + does it work the way it's supposed to?

In [ ]:
'''
idx = learning_df[learning_df['group_code'] == 0].index
X_ref_train = learning_df.drop(index=idx).drop(['virus', 'host', 'group', 'group_code', 'y'], axis=1).values
X_ref_test = learning_df.iloc[idx,:].drop(['virus', 'host', 'group', 'group_code', 'y'], axis=1).values
y_ref_train = learning_df.drop(index=idx).iloc[:, -1]
y_ref_test = learning_df.iloc[idx,-1]
'''

In [ ]:
'''
model = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state=1)
model.fit(X_ref_train, y_ref_train)
y_pred = model.predict(X_ref_test)
'''

In [ ]:
'''
from sklearn.metrics import f1_score
print(f'F1 is: {f1_score(y_ref_test, y_pred)}')
'''

Determining most certain class 1 predictions done with a dict

In [ ]:
# determining most certain class 1 predictions done with a dict
'''
# define the function
def write_down(x):
    if x['virus'] not in best_hosts.keys(): 
        best_hosts[x['virus']] = {} 
        best_hosts[x['virus']]['hosts'] = []
    best_hosts[x['virus']]['hosts'].append((x['host'], x['1']))

# apply the function
best_hosts = {}
best_hosts_df = pd.DataFrame(columns=['virus', 'host', 'prob'])
_ = df_all[mask].apply(write_down, axis=1)
del _

# write down the results to file 
with open('host_predictions_1_based_log.tsv', 'w') as of:
    of.write('virus\tvirus_name\thost\thost_name\t1\n')
    for key, val in best_hosts.items():
        for el in val["hosts"]:
            of.write(f'{key}\t{orgs["virus"][key]["organism_name"].split(",")[0]}\t{el[0]}\t{orgs["host"][el[0]]["organism_name"]}\t{el[1]}\n')
'''

Old version just to check if both return the same things

In [ ]:
# Old version of getting the max subgroups - inefficient
'''
# WATCH OUT - long (approx. 3 min.)
viruses = pd.unique(df_all['virus'])
best_hosts = {}
# best_hosts_df = pd.DataFrame(columns=['virus', 'host', '1'])

for vir in viruses:
    best_hosts[vir] = {}
    best_hosts[vir]['hosts'] = []
    mask = df_all['virus'] == vir
    vir_max_prob = max(df_all[mask]['1'])

    # best_hosts_df = pd.concat([best_hosts_df, correct_df.loc[
    #         (mask) & 
    #         (correct_df['prob'] == vir_max_prob),  
    #         ['virus', 'host', 'prob']]])

    for _, row in df_all.loc[(mask) & (df_all['1'] == vir_max_prob),].iterrows():
        best_hosts[vir]['hosts'].append((row['host'], row['1']))

# writing down the results
with open('2_log_test.tsv', 'w') as of:
    of.write('virus\tvirus_name\thost\thost_name\t1\n')
    for key, val in best_hosts.items():
        for el in val["hosts"]:
            of.write(f'{key}\t{orgs["virus"][key]["organism_name"].split(",")[0]}\t{el[0]}\t{orgs["host"][el[0]]["organism_name"]}\t{el[1]}\n')
'''
